__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
# avito_path = 'sevastopol/kvartiry/prodam'
avito_path = 'sevastopol/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-13 13:55:29 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-13 13:55:29 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-13 13:55:31 | AdsListParser: downloader init
[INFO    ] 2022-09-13 13:55:31 | AvitoParser: init
[INFO    ] 2022-09-13 13:55:37 | AvitoParser: 23 pages for read
[INFO    ] 2022-09-13 13:55:37 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-13 13:55:41 | AdsListParser: read page 1
[INFO    ] 2022-09-13 13:55:45 | AdsListParser: read page 2
[INFO    ] 2022-09-13 13:55:48 | AdsListParser: read page 3
[INFO    ] 2022-09-13 13:55:52 | AdsListParser: read page 4
[INFO    ] 2022-09-13 13:55:54 | AdsListParser: read page 5
[INFO    ] 2022-09-13 13:55:57 | AdsListParser: read page 6
[INFO    ] 2022-09-13 13:56:00 | AdsListParser: read page 7
[INFO    ] 2022-09-13 13:56:05 | AdsListParser: read page 8
[INFO    ] 2022-09-13 13:56:08 | AdsListParser: read page 9
[INFO    ] 2022-09-13 13:56:13 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1121
1111


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
333,2302356440,Объявление «Участок 10 сот. (ИЖС)» 32 фотографии,1800000,,"Севастополь, Балаклавский район, Орлиновский муниципальный округ","Участки по новому ген плану войдут в зону населенного пункта и под малоэтажную жилищную застройку(ижс) на противоположном берегу большого озера у села Широкое, в 300 метрах от берега, прекрасные обзорные виды и чистейший воздух. \n\nЧитаем внимательно! \nНа с",7,2022-09-13 13:56:54.600362
872,2365205749,Объявление «Участок 5 сот. (ИЖС)» 13 фотографий,4200000,,Монастырское ш.,"Предлагается к продаже участок,5 соток, на Фиоленте-в первой линии от дороги (ориентир-конечная остановка автобусов №3, где воинская часть или Яшмовый пляж). Участок прямоугольной формы, одна половина-идеально ровная, другая-под небольшим уклоном. Коммуник",18,2022-09-13 13:56:54.600362
